# Environment

In [ ]:
!pip install -r /content/drive/MyDrive/text_mining/multilingual-summarization-20250312/requirements.txt

In [ ]:
!wandb login

wandb: Currently logged in as: kazetof (llm-summarization-fbt3r5) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from huggingface_hub import login

# Replace with your actual Hugging Face token
hf_token = ""

# Log in to Hugging Face
login(token=hf_token)

# mbart

In [ ]:
import os

import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import load_dataset, load_from_disk
from transformers import Trainer, TrainingArguments

# 使用するモデルとトークナイザーのロード
model_name = "facebook/mbart-large-50"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# データセットの読み込み

dataset_path = "/content/drive/MyDrive/text_mining/hf_dataset_from_quiscale"
language = "de"
dataset = load_from_disk(os.path.join(dataset_path, language))
train_dataset = dataset["train"]
val_dataset = dataset["validation"]


In [ ]:
def generate_summary(text: str, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(model.device)
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# summarize sample text
sample_text = "Your article text here."
print(generate_summary(sample_text, model, tokenizer))

Your article text here.


In [ ]:
from rouge import Rouge
import numpy as np

def calculate_rouge_scores(generated, reference):
    rouge = Rouge()
    scores = rouge.get_scores(generated, reference)[0]

    return {
        "rouge-1": scores["rouge-1"]["f"],
        "rouge-2": scores["rouge-2"]["f"],
        "rouge-l": scores["rouge-l"]["f"],
    }


In [ ]:

def calculate_avg_rouge_scores(val_dataset, model, tokenizer):
  max_samples = 10
  indices = list(range(len(val_dataset)))
  sample_indices = np.random.choice(
      indices,
      min(max_samples, len(val_dataset)),
      replace=False
  )

  rouge_scores = []
  for idx in sample_indices:
      print("idx : ", idx)
      text = val_dataset[int(idx)]["text"]
      generated = generate_summary(text, model, tokenizer)
      reference = val_dataset[int(idx)]["summary"]

      score = calculate_rouge_scores(generated, reference)
      rouge_scores.append(score)
      print(score)

  # Calculate average scores
  avg_rouge_scores = {
      metric: np.mean([score[metric] for score in rouge_scores])
      for metric in ["rouge-1", "rouge-2", "rouge-l"]
  }
  return avg_rouge_scores

In [ ]:
avg_rouge_scores = calculate_avg_rouge_scores(val_dataset, model, tokenizer)
print("avg : ", avg_rouge_scores)

idx :  305
{'rouge-1': 0.11881187629840231, 'rouge-2': 0.0, 'rouge-l': 0.09900989610038256}
idx :  484
{'rouge-1': 0.21487602830680974, 'rouge-2': 0.07594936251241816, 'rouge-l': 0.21487602830680974}
idx :  131
{'rouge-1': 0.14285713860102053, 'rouge-2': 0.035928139721037415, 'rouge-l': 0.12857142431530627}
idx :  460
{'rouge-1': 0.16071428072863536, 'rouge-2': 0.04411764205990541, 'rouge-l': 0.1428571378714925}
idx :  499
{'rouge-1': 0.15053763047751195, 'rouge-2': 0.03773584545211854, 'rouge-l': 0.10752687778933995}
idx :  85
{'rouge-1': 0.12389380078941202, 'rouge-2': 0.0, 'rouge-l': 0.10619468574516426}
idx :  72
{'rouge-1': 0.25954197983800487, 'rouge-2': 0.1052631530193908, 'rouge-l': 0.24427480426548578}
idx :  5
{'rouge-1': 0.2409638511649006, 'rouge-2': 0.048309174841886945, 'rouge-l': 0.22891565839381628}
idx :  96
{'rouge-1': 0.03669724328255251, 'rouge-2': 0.0, 'rouge-l': 0.03669724328255251}
idx :  145
{'rouge-1': 0.15602836444846852, 'rouge-2': 0.02439023957168429, 'rouge

In [ ]:
def tokenize_function(examples):
    inputs = examples["text"]
    targets = examples["summary"]

    # tokenize input and label
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=150)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# apply tokenizer for dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# setting for training
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=200,
    evaluation_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    fp16=True,
    load_best_model_at_end=True
)

# setting Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

# finetuning
trainer.train()

# save model
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")




Map:   0%|          | 0/3999 [00:00<?, ? examples/s]

Map:   0%|          | 0/501 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-ff488d5eb6ec>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kazetof (llm-summarization-fbt3r5) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
200,2.373800,1.880854
400,5.637900,11.790870
600,6.250200,1.791905
800,3.719400,8.577353
1000,2.877300,1.817305
1200,1.489800,1.902968
1400,1.394900,1.746343


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
Could not locate the best model at ./results/checkpoint-1400/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/sentencepiece.bpe.model',
 './finetuned_model/added_tokens.json',
 './finetuned_model/tokenizer.json')

In [ ]:
avg_rouge_scores_finetuned = calculate_avg_rouge_scores(val_dataset, model, tokenizer)
print("avg : ", avg_rouge_scores_finetuned)

idx :  460
{'rouge-1': 0.3962264100943397, 'rouge-2': 0.11764705382461094, 'rouge-l': 0.35849056103773586}
idx :  406
{'rouge-1': 0.5591397805757892, 'rouge-2': 0.4102564058119659, 'rouge-l': 0.5591397805757892}
idx :  206
{'rouge-1': 0.47787610130785496, 'rouge-2': 0.21311474926095147, 'rouge-l': 0.44247787121935944}
idx :  497
{'rouge-1': 0.4277456601089245, 'rouge-2': 0.20720720270432602, 'rouge-l': 0.41618496646730596}
idx :  67
{'rouge-1': 0.3749999953719009, 'rouge-2': 0.19178081723316875, 'rouge-l': 0.36363635900826446}
idx :  124
{'rouge-1': 0.1777777727780523, 'rouge-2': 0.024691353031551073, 'rouge-l': 0.1777777727780523}
idx :  152
{'rouge-1': 0.4337349349448396, 'rouge-2': 0.1826923030810837, 'rouge-l': 0.38554216386050233}
idx :  362
{'rouge-1': 0.30630630137488846, 'rouge-2': 0.0965517191438766, 'rouge-l': 0.2522522473208344}
idx :  493
{'rouge-1': 0.4258064468129033, 'rouge-2': 0.2030456803875391, 'rouge-l': 0.39999999520000007}
idx :  448
{'rouge-1': 0.31481480981652954

debug japanese

In [ ]:
text = val_dataset[0]["text"]
# ref = val_dataset[0]["summary"]
ref = "株式会社ナムコは、1955年に有限会社中村製作所として設立され、1977年に社名をナムコに変更した日本のゲームメーカーおよびアミューズメント施設運営企業であった。1980年代には『パックマン』など多くのヒットゲームを生み出し、その後も『鉄拳』や『リッジレーザー』など多くの人気ゲームシリーズを発表した。2005年にバンダイと経営統合し、2006年にバンダイナムコゲームスに社名変更した。アミューズメント部門は2006年に独立し、2018年にバンダイナムコアミューズメントに社名変更した。ナムコは日本のゲーム産業史上重要な役割を果たした。そのブランドは現在でも後継企業で使用されている。"
summary = generate_summary(text, model, tokenizer)

In [ ]:
print(summary)

株式会社ナムコは、バンダイナムコエンターテインメントとバンダイナムコアミューズメントの本社を運営するゲームセンター「namco」を展開する企業。1988年に設立され、1991年9月に上場。2005年に廃止。本社は東京都大田区矢口2丁目1番21号で、本社ビルは東京都大田区矢口の本社ビル。2021年にマンションが建設された。


In [ ]:
print(ref)

株式会社ナムコは、1955年に有限会社中村製作所として設立され、1977年に社名をナムコに変更した日本のゲームメーカーおよびアミューズメント施設運営企業であった。1980年代には『パックマン』など多くのヒットゲームを生み出し、その後も『鉄拳』や『リッジレーザー』など多くの人気ゲームシリーズを発表した。2005年にバンダイと経営統合し、2006年にバンダイナムコゲームスに社名変更した。アミューズメント部門は2006年に独立し、2018年にバンダイナムコアミューズメントに社名変更した。ナムコは日本のゲーム産業史上重要な役割を果たした。そのブランドは現在でも後継企業で使用されている。


In [ ]:
calculate_rouge_scores("株式会社ナムコ", "株式会社ナムコは")

{'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}

In [ ]:
calculate_rouge_scores(summary, ref)

{'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}